reference:
- https://github.com/benitomartin/Milvus-Agentic-RAG-Claude/blob/main/Agentic_RAG_Using_Claude_3_5_Sonnet%2C_LlamaIndex%2C_and_Milvus.ipynb

- https://zilliz.com/blog/a-beginners-guide-to-using-llama-3-with-ollama-milvus-langchain

- https://github.com/tspannhw/AIM-NYCStreetCams/blob/main/MultipleVectorsAdvanced%20SearchDataModelDesign/streetcamsrag.ipynb

- https://github.com/dhivyeshrk/Retrieval-Augmented-Generation-for-news

- https://docs.llamaindex.ai/en/stable/examples/vector_stores/MilvusIndexDemo/


In [ ]:
# !pip install openai

In [ ]:
# !pip install llama_index

In [ ]:
# !pip install llama-index-vector-stores-milvus

In [ ]:
# !pip install pymilvus

In [1]:
import openai
from llama_index.llms.openai import OpenAI
from llama_index.core import SimpleDirectoryReader
from llama_index.core import VectorStoreIndex, StorageContext
from llama_index.vector_stores.milvus import MilvusVectorStore

In [2]:
openai.api_key = "your key"

### Read files to documents

In [29]:
# from google.colab import drive
# drive.mount('/content/drive')

In [3]:
documents = SimpleDirectoryReader(
    input_dir="/Users/elinazhuang/Desktop/wsj_until2019_0928"
).load_data()

In [4]:
print("Document ID 0:", documents[0].doc_id)
print("Document ID 1:", documents[1].doc_id)

Document ID 0: 0c642837-4087-419a-871d-ed6df4a2ed86
Document ID 1: 7025930d-7e24-40c5-9d25-8bf2fce83525


### Data indexing

In [5]:
vector_store = MilvusVectorStore(uri="/Users/elinazhuang/Desktop/milvus_wsj.db", dim=1536, overwrite=True)
storage_context = StorageContext.from_defaults(vector_store=vector_store)
index = VectorStoreIndex.from_documents(documents, storage_context=storage_context)

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/pydantic/_internal/_fields.py:132: UserWarning: Field "model_name" in HuggingFaceInferenceAPIEmbeddings has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(


### Query

In [7]:
from llama_index.llms.openai import OpenAI
# llm = OpenAI(model="gpt-3.5-turbo")
llm = OpenAI(model="gpt-4o-mini", strict=True)

In [8]:
query_engine = index.as_query_engine(similarity_top_k=10,response_mode="simple_summarize",
    verbose=True,llm=llm)

In [9]:
# llm = OpenAI(model="gpt-4o")
llm = OpenAI(model="gpt-4o")

query_engine = index.as_query_engine(similarity_top_k=10,response_mode="tree_summarize",verbose=True,llm=llm)

# Test for RAG vector db information retrieval
res = query_engine.query("Give me the news on 2016-4-26")
print(res)

The news on April 26, 2016, includes:

1. "Portman Won’t Run for President in 2016" - Politics
2. "Artist Richard Prince’s Secret Retreat" - Feature
3. "Richard Prince’s Private Marfa" - Slideshow
4. "Fendi and Dimore Studio Team Up for Design Miami" - Partnership
5. "Roger Moore on Frank Sinatra’s ‘One for My Baby’" - Playlist
6. "Somali Militants Kill 36 in Kenya Attack" - Africa
7. "CEO Council Members and Guests" - Leadership
8. "A Retired Musician Finds a New ‘Gig’" - Wealth Management
9. "Germany Expected to Approve Tax Incentives for Electric Cars" - World
10. "Brazil’s Central Bank Seen Raising Benchmark Rate Again" - World
11. "A Connecticut Villa" - House of the Day
12. "Suspect in W.Va. Killings Found Dead" - U.S.
13. "Sprint Escalates Price War With Half-Off Bills" - Tech
14. "Consumer Group Seeks Adviser Horror Stories" - Wealth Management
15. "Delta Revenue Measure Rose in November" - Business
16. "Video Claims Islamic State Responsibility for Saudi Shooting" - World
17. 

In [12]:
query_engine = index.as_query_engine(similarity_top_k=10,response_mode="tree_summarize",verbose=True,llm=llm)
res = query_engine.query("""
### Role ###
You are a highly knowledgeable and helpful finance economist specializing in the US equity market. 

### Task ###
Please note the following:
- Important information is enclosed within `<>`.
- Fill in the details in the provided placeholders `()`.
- Market Regime is defined as: a period of time when a market's conditions remain similar and fluctuate, affecting key investing factors. These factors include risk/return relationships, correlations, and volatilities
Your task is to analyze the news titles and come up with 5 market regimes, represented by a regime number (0, 1, 2, 3, 4). For each market regime, provide 10 <US Market-related News> <across different time periods> as evidence to support your analysis, use as many news as necessary.
Try to use <DIFFERENT DATES OF NEWS FOR DIFFERENT MARKET REGIMES>.

### Output Format ###
**Market Regime:** 0
**Summary of Market Regime:** (In less than 3 sentences)
**Evidence:** (Provide one paragraph with the selected news as evidence to support your reasoning behind the identified market regime.)
**References / titles and dates to the News:** ()

**Market Regime:** 1
**Summary of Market Regime:** (In less than 3 sentences)
**Evidence:** (Provide one paragraph with the selected news as evidence to support your reasoning behind the identified market regime.)
**References / titles and dates to the News:** ()

**Market Regime:** 2
**Summary of Market Regime:** (In less than 3 sentences)
**Evidence:** (Provide one paragraph with the selected news as evidence to support your reasoning behind the identified market regime.)
**References / titles and dates to the News:** ()

**Market Regime:** 3
**Summary of Market Regime:** (In less than 3 sentences)
**Evidence:** (Provide one paragraph with the selected news as evidence to support your reasoning behind the identified market regime.)
**References / titles and dates to the News:** ()

**Market Regime:** 4
**Summary of Market Regime:** (In less than 3 sentences)
**Evidence:** (Provide one paragraph with the selected news as evidence to support your reasoning behind the identified market regime.)
**References / titles and dates to the News:** ()

""")
print(res)

**Market Regime:** 0
**Summary of Market Regime:** This regime is characterized by significant regulatory changes and their impact on financial markets and institutions.
**Evidence:** During this period, there were numerous regulatory shifts affecting the financial sector. The CFTC Chairman's readiness to revamp swaps rules and the SEC's charges against a former SAP America executive for insider trading highlight the regulatory scrutiny. Additionally, the rollback of net-neutrality rules and the Senate's vote to undo a rule aimed at helping private-sector workers save for retirement further underscore the regulatory environment's influence on market dynamics.
**References / titles and dates to the News:** 
- "CFTC Chairman Readies Swaps-Rules Revamp" (2017-08-11)
- "SEC Charges Former SAP America Executive With Insider Trading" (2016-06-16)
- "FCC Chairman Outlines Net-Neutrality Rollback" (2017-04-27)
- "Senate Votes to Undo Rule Aimed at Helping Private-Sector Workers Save for Retire

In [21]:
# llm = OpenAI(model="gpt-4o")
llm = OpenAI(model="gpt-4o")

query_engine = index.as_query_engine(similarity_top_k=10,response_mode="tree_summarize",verbose=True,llm=llm)
res = query_engine.query("""
### Role ###
You are a highly knowledgeable and helpful finance economist specializing in the US equity market. 

### Task ###
Please note the following:
- Important information is enclosed within `<>`.
- Fill in the details in the provided placeholders `()`.
- Market Regime is defined as: a period of time when a market's conditions remain similar and fluctuate, affecting key investing factors. These factors include risk/return relationships, correlations, volatilities, interest rate, and inflation.
- Also consider the stock market/economic cycle: Recovery, Expansion, Peak, Bull, Contraction, Recession, Bear.
Your task is to analyze the news titles and come up with 3 market regimes, represented by a regime number (0, 1, 2). For each market regime, provide 10 <US Market-related News> <across different time periods> as evidence to support your analysis, use as many news as necessary.
Try to use <DIFFERENT DATES OF NEWS FOR DIFFERENT MARKET REGIMES>.

### Output Format ###
**Market Regime:** 0
**Keyword of Market Regime:** (Use a word to describe the market regime)
**Summary of Market Regime:** (In less than 3 sentences)
**Evidence:** (Provide one paragraph with the selected news and dates as evidence to support your reasoning behind the identified market regime.)
**References / titles and dates to the News:** ()

**Market Regime:** 1
**Keyword of Market Regime:** (Use a word to describe the market regime)
**Summary of Market Regime:** (In less than 3 sentences)
**Evidence:** (Provide one paragraph with the selected news and dates as evidence to support your reasoning behind the identified market regime.)
**References / titles and dates to the News:** ()

**Market Regime:** 2
**Keyword of Market Regime:** (Use a word to describe the market regime)
**Summary of Market Regime:** (In less than 3 sentences)
**Evidence:** (Provide one paragraph with the selected news as evidence to support your reasoning behind the identified market regime.)
**References / titles and dates to the News:** ()

""")
print(res)

**Market Regime:** 0
**Keyword of Market Regime:** Expansion
**Summary of Market Regime:** This regime is characterized by robust economic growth, increased consumer confidence, and rising corporate earnings, leading to a bullish stock market.
**Evidence:** During this period, several indicators pointed towards economic expansion. For instance, "South Korea’s Economy Grew 0.9% in First Quarter" and "Singapore Central Bank Predicts Steady Economic Growth in 2017" highlight strong economic performance in Asia, which often correlates with positive sentiment in the US markets. Additionally, "U.S. Durable Orders Edged Up In March" and "U.S. Retail Sales Fall 2.4% in February - Redbook" indicate mixed but generally positive economic data, supporting the notion of an expanding economy. The "German Consumer Sentiment Set to Leap in May" and "China’s Industrial Profits Growth Slows in March" further illustrate global economic strength, which typically benefits US equities.
**References / titles

In [22]:
llm = OpenAI(model="gpt-4o")

query_engine = index.as_query_engine(similarity_top_k=10,response_mode="tree_summarize",verbose=True,llm=llm)
res = query_engine.query("""
### Role ###
You are a highly knowledgeable and helpful finance economist specializing in the US equity market. 

### Task ###
Please note the following:
- Important information is enclosed within `<>`.
- Fill in the details in the provided placeholders `()`.
- Market Regime is defined as: a period of time when a market's conditions remain similar and fluctuate, affecting key investing factors. These factors include risk/return relationships, correlations, volatilities, interest rate, and inflation.
- Also consider the stock market/economic cycle: Recovery, Expansion, Peak, Contraction, and Recession.
You have access to <all relevant news with matched regime groups>,please reference on US-specific news <accross different time span>.
Your task is to analyze and explain all the market regimes, represented by a regime number (0, 1, 2, 3, 4). For each market regime, provide 5 pieces of <US Market-related News> as evidence to support your analysis, Use as many news as necessary.

### Output Format ###
**Market Regime:** 0
**Keyword of Market Regime:** (Use a word to describe the market regime)
**Summary of Market Regime:** (In less than 3 sentences)
**Evidence:** (Provide one paragraph with the selected news and dates as evidence to support your reasoning behind the identified market regime.)
**References / Links and date to the News:** ()

**Market Regime:** 1
**Keyword of Market Regime:** (Use a word to describe the market regime)
**Summary of Market Regime:** (In less than 3 sentences)
**Evidence:** (Provide one paragraph with the selected news and dates as evidence to support your reasoning behind the identified market regime.)
**References / Links and date to the News:** ()

**Market Regime:** 2
**Keyword of Market Regime:** (Use a word to describe the market regime)
**Summary of Market Regime:** (In less than 3 sentences)
**Evidence:** (Provide one paragraph with the selected news and dates as evidence to support your reasoning behind the identified market regime.)
**References / Links and date to the News:** ()

**Market Regime:** 3
**Keyword of Market Regime:** (Use a word to describe the market regime)
**Summary of Market Regime:** (In less than 3 sentences)
**Evidence:** (Provide one paragraph with the selected news and dates as evidence to support your reasoning behind the identified market regime.)
**References / Links and date to the News:** ()

**Market Regime:** 4
**Keyword of Market Regime:** (Use a word to describe the market regime)
**Summary of Market Regime:** (In less than 3 sentences)
**Evidence:** (Provide one paragraph with the selected news and dates as evidence to support your reasoning behind the identified market regime.)
**References / Links and date to the News:** ()

""")
print(res)


**Market Regime:** 0
**Keyword of Market Regime:** Recovery
**Summary of Market Regime:** This regime is characterized by the initial phase of economic improvement following a recession, marked by increasing economic activity and improving investor sentiment.
**Evidence:** During this period, the U.S. economy shows signs of recovery with positive economic data and corporate earnings. For instance, "U.S. Durable Orders Edged Up In March" (April 27, 2017) indicates a rise in durable goods orders, a sign of increasing business investment. Additionally, "U.S. Retail Sales Fall 2.4% in February - Redbook" (March 1, 2016) reflects a temporary setback in consumer spending, but overall trends suggest recovery. The "U.S. 10-Year Government Bond Yield Falls" (June 16, 2016) points to lower borrowing costs, supporting economic growth. "Small-Business Confidence Improves in April, but Remains Lackluster" (May 10, 2016) highlights improving sentiment among small businesses, a key driver of job crea